<a href="https://colab.research.google.com/github/Noertri/scraper-nusabali/blob/main/scraper_nusabali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Menginstall package

1. upload file requirements.txt
2. jalankan sel dibawah ini

In [ ]:
!pip install --upgrade -r requirements.txt

# Program

In [3]:
import httpx
from bs4 import BeautifulSoup
from dataclasses import dataclass, asdict
import pandas as pd
from urllib import parse
import dateparser
from datetime import datetime


client = httpx.Client(follow_redirects=True, timeout=5.)
main_url = "https://www.nusabali.com"
results = []


@dataclass
class Result:
    tanggal: str = ""
    judul: str = ""
    url: str = ""


def request_main_page(keyword):
    page_url = parse.urljoin(main_url, "search")

    params = {
        'keyword': f'{keyword}',
    }

    query_str = parse.urlencode(params)

    referer_url = parse.urlunsplit(("https", "www.nusabali.com", "search", query_str, ""))

    headers = {
        'authority': 'www.nusabali.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'id-ID,id;q=0.9,en-US;q=0.8,en;q=0.7,ms;q=0.6,ja;q=0.5',
        'dnt': '1',
        'referer': f'{referer_url}',
        'sec-ch-ua': '"Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    }


    response = client.get(page_url, params=params, headers=headers)

    if response.status_code < 400:
        return response


# scrape halaman awal
r = request_main_page(keyword="bali")
soup = BeautifulSoup(r.text, "html.parser")
article_list = soup.select("div#article-list>div.row")
if article_list:
    for article in article_list:
        h5_tag = article.select_one("div.col-xs-8.col-md-8 h5")
        span_tag = article.select_one("div.col-xs-12.entry-metas>span>span")
        article_title = h5_tag.get_text(strip=True, separator=" ") if h5_tag is not None else ""
        article_url = a_tag.get("href", "") if (a_tag := h5_tag.select_one("a")) is not None else ""
        article_date = span_tag.get_text(strip=True, separator=" ") if span_tag is not None else ""
        article_date = datetime.strptime(article_date, "%d %b %Y %H:%M")

        result = Result(tanggal=article_date.strftime("%d %B %Y %H:%M"), judul=article_title, url=article_url)
        results.append(asdict(result))



# Menyimpan hasil ke file excel

In [19]:
def save_to_excel(prefix):
    filename = "{0}_{1}.xlsx".format(prefix, datetime.now().strftime("%d%m%Y%H%M%S"))

    df = pd.DataFrame(results)

    print(f"Menyimpan ke file {filename}...")

    try:
        df.to_excel(filename, index=False)
        print("Berhasil!!!")
    except Exception as err:
        print(err)

save_to_excel("artikel_kependudukan")

Menyimpan ke file artikel_kependudukan_01082023040210.xlsx...
Berhasil!!!
